In [ ]:
using VortexCollisions

In [ ]:
using PyPlot

In [ ]:
Δt = 2E-6;
nt = 1000;

In [ ]:
M = 64;
N = 64;

In [ ]:
Mcut = M;
Ncut = N;

In [ ]:
gr = Grid2d(M,N)

In [ ]:
ft = FourierTransform(gr, mcut=Mcut, ncut=Ncut)

In [ ]:
op = FokkerPlanckOperator(gr, ft)

In [ ]:
const a = 1;
const b = 2;

In [ ]:
function u_init(x,y)
    exp(a*cos(x-π) + b*cos(y-π))
end

In [ ]:
u₀ = get_field(gr);

In [ ]:
for j in 1:size(u₀,2)
    for i in 1:size(u₀,1)
         u₀[i,j]  = u_init(gr.x[i], gr.y[j])
    end
end

In [ ]:
figure()
contour(u₀, colors="k")
pcolormesh(u₀)
colorbar()

In [ ]:
f₀ = zeros(u₀);

In [ ]:
collision_operator!(op, u₀, f₀);

In [ ]:
figure()
pcolormesh(f₀)
colorbar()

In [ ]:
u = Vector{Matrix{eltype(u₀)}}(nt+1)
for n in 1:length(u)
    u[n] = zeros(u₀)
end
u[1] .= u₀;

In [ ]:
for n in 1:nt
    timestep!(op, u[n], u[n+1], Δt)
end

In [ ]:
nplot = nt+1
u₁ = u[nplot];

In [ ]:
figure(figsize=(12,5))

subplot(121)
contour(u₀, colors="k")
pcolormesh(u₀)
colorbar()
title("Initial Condition")

subplot(122)
contour(u₁, colors="k")
pcolormesh(u₁)
colorbar()
title("Final Solution")

In [ ]:
# compute energy
h₀ = get_field(gr);
h₁ = get_field(gr);

In [ ]:
uhat = get_trans(ft)
hhat = get_trans(ft)

prfft!(ft, u₀, uhat)
hhat .= op.Δ⁻¹ .* uhat
irfft!(ft, hhat, h₀)

prfft!(ft, u₁, uhat)
hhat .= op.Δ⁻¹ .* uhat
irfft!(ft, hhat, h₁)
;

In [ ]:
figure(figsize=(12,5))

subplot(121)
scatter(u₀[:], h₀[:])
title("Initial Condition")
xlabel("ω")
ylabel("h")

subplot(122)
scatter(u₁[:], h₁[:])
title("Final Solution")
xlabel("ω")
ylabel("h")